# Object Detection Demo
Welcome to the object detection inference walkthrough!  This notebook will walk you step by step through the process of using a pre-trained model to detect objects in an image. Make sure to follow the [installation instructions](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/installation.md) before you start.

# Imports

In [1]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile
import json

from distutils.version import StrictVersion
from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image

# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")
from object_detection.utils import ops as utils_ops

if StrictVersion(tf.__version__) < StrictVersion('1.12.0'):
  raise ImportError('Please upgrade your TensorFlow installation to v1.12.*.')


## Env setup

In [2]:
# This is needed to display the images.
%matplotlib inline

## Object detection imports
Here are the imports from the object detection module.

In [3]:
from utils import label_map_util

from utils import visualization_utils as vis_util

# Model preparation 

## Variables

Any model exported using the `export_inference_graph.py` tool can be loaded here simply by changing `PATH_TO_FROZEN_GRAPH` to point to a new .pb file.  

By default we use an "SSD with Mobilenet" model here. See the [detection model zoo](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md) for a list of other models that can be run out-of-the-box with varying speeds and accuracies.

In [4]:
# What model to download.
MODEL_NAME = 'ssd_mobilenet_v1_coco_2017_11_17'
MODEL_FILE = MODEL_NAME + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'

# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_FROZEN_GRAPH = MODEL_NAME + '/frozen_inference_graph.pb'

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = os.path.join('data', 'mscoco_label_map.pbtxt')

## Download Model

In [5]:
opener = urllib.request.URLopener()
opener.retrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)
tar_file = tarfile.open(MODEL_FILE)
for file in tar_file.getmembers():
  file_name = os.path.basename(file.name)
  if 'frozen_inference_graph.pb' in file_name:
    tar_file.extract(file, os.getcwd())

## Load a (frozen) Tensorflow model into memory.

In [6]:
detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.GraphDef()
  with tf.gfile.GFile(PATH_TO_FROZEN_GRAPH, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')

## Loading label map
Label maps map indices to category names, so that when our convolution network predicts `5`, we know that this corresponds to `airplane`.  Here we use internal utility functions, but anything that returns a dictionary mapping integers to appropriate string labels would be fine

In [7]:
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

## Helper code

In [8]:
def load_image_into_numpy_array(image):
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

# Detection

In [9]:
# For the sake of simplicity we will use only 2 images:
# image1.jpg
# image2.jpg
# If you want to test the code with your images, just add path to the images to the TEST_IMAGE_PATHS.
PATH_TO_TEST_IMAGES_DIR = '/home/ubuntu/inzamam/videoframes'
TEST_IMAGE_PATHS = [ os.path.join(PATH_TO_TEST_IMAGES_DIR, 'frame{:06d}.jpg'.format(i)) for i in range(0, 299) ]

# Size, in inches, of the output images.
IMAGE_SIZE = (12, 8)

In [10]:
print(TEST_IMAGE_PATHS)

['/home/ubuntu/inzamam/videoframes/frame000000.jpg', '/home/ubuntu/inzamam/videoframes/frame000001.jpg', '/home/ubuntu/inzamam/videoframes/frame000002.jpg', '/home/ubuntu/inzamam/videoframes/frame000003.jpg', '/home/ubuntu/inzamam/videoframes/frame000004.jpg', '/home/ubuntu/inzamam/videoframes/frame000005.jpg', '/home/ubuntu/inzamam/videoframes/frame000006.jpg', '/home/ubuntu/inzamam/videoframes/frame000007.jpg', '/home/ubuntu/inzamam/videoframes/frame000008.jpg', '/home/ubuntu/inzamam/videoframes/frame000009.jpg', '/home/ubuntu/inzamam/videoframes/frame000010.jpg', '/home/ubuntu/inzamam/videoframes/frame000011.jpg', '/home/ubuntu/inzamam/videoframes/frame000012.jpg', '/home/ubuntu/inzamam/videoframes/frame000013.jpg', '/home/ubuntu/inzamam/videoframes/frame000014.jpg', '/home/ubuntu/inzamam/videoframes/frame000015.jpg', '/home/ubuntu/inzamam/videoframes/frame000016.jpg', '/home/ubuntu/inzamam/videoframes/frame000017.jpg', '/home/ubuntu/inzamam/videoframes/frame000018.jpg', '/home/ubun

In [18]:
with detection_graph.as_default():
    ops = tf.get_default_graph().get_operations()
    all_tensor_names = {output.name for op in ops for output in op.inputs}
    for x in all_tensor_names:
        print(x)

Postprocessor/BatchMultiClassNonMaxSuppression/map/while/MultiClassNonMaxSuppression/ClipToWindow_23/concat:0
Postprocessor/BatchMultiClassNonMaxSuppression/map/while/MultiClassNonMaxSuppression/ClipToWindow_52/Maximum:0
MultipleGridAnchorGenerator/Meshgrid_5/ExpandedShape/concat/_154__cf__157:0
Postprocessor/BatchMultiClassNonMaxSuppression/map/while/PadOrClipBoxList/cond_1/cond/Switch:0
Postprocessor/BatchMultiClassNonMaxSuppression/map/while/MultiClassNonMaxSuppression/ClipToWindow_60/split:1
Postprocessor/BatchMultiClassNonMaxSuppression/map/while/MultiClassNonMaxSuppression/ClipToWindow_56/Maximum_3:0
Postprocessor/BatchMultiClassNonMaxSuppression/map/while/MultiClassNonMaxSuppression/ClipToWindow_48/Where:0
Postprocessor/BatchMultiClassNonMaxSuppression/map/while/MultiClassNonMaxSuppression/Minimum_51:0
Postprocessor/BatchMultiClassNonMaxSuppression/map/while/MultiClassNonMaxSuppression/ClipToWindow_48/concat:0
Postprocessor/BatchMultiClassNonMaxSuppression/map/while/MultiClassNo

Postprocessor/BatchMultiClassNonMaxSuppression/map/while/MultiClassNonMaxSuppression/Reshape_77/shape:0
Postprocessor/BatchMultiClassNonMaxSuppression/map/while/MultiClassNonMaxSuppression/ClipToWindow_48/unstack:3
Postprocessor/BatchMultiClassNonMaxSuppression/map/while/MultiClassNonMaxSuppression/ClipToWindow_53/split:0
Postprocessor/BatchMultiClassNonMaxSuppression/map/while/MultiClassNonMaxSuppression/non_max_suppression_1/NonMaxSuppressionV2:0
Postprocessor/BatchMultiClassNonMaxSuppression/map/while/MultiClassNonMaxSuppression/stack_26/1:0
Postprocessor/BatchMultiClassNonMaxSuppression/map/while/MultiClassNonMaxSuppression/ClipToWindow_81/unstack:2
Postprocessor/BatchMultiClassNonMaxSuppression/map/while/MultiClassNonMaxSuppression/zeros_like_75:0
MultipleGridAnchorGenerator/assert_equal/Assert/Assert/data_1:0
Postprocessor/BatchMultiClassNonMaxSuppression/map/while/MultiClassNonMaxSuppression/Gather_7/Gather_2:0
Postprocessor/BatchMultiClassNonMaxSuppression/map/while/MultiClassN

FeatureExtractor/MobilenetV1/MobilenetV1/Conv2d_13_depthwise/depthwise:0
Postprocessor/BatchMultiClassNonMaxSuppression/map/while/MultiClassNonMaxSuppression/ClipToWindow_49/Minimum_3:0
Postprocessor/BatchMultiClassNonMaxSuppression/map/while/MultiClassNonMaxSuppression/strided_slice_23/stack:0
Postprocessor/BatchMultiClassNonMaxSuppression/map/while/MultiClassNonMaxSuppression/ClipToWindow_59/split:3
Postprocessor/BatchMultiClassNonMaxSuppression/map/while/MultiClassNonMaxSuppression/ClipToWindow_81/Maximum_3:0
Postprocessor/BatchMultiClassNonMaxSuppression/map/while/MultiClassNonMaxSuppression/Reshape_39:0
Postprocessor/BatchMultiClassNonMaxSuppression/map/while/MultiClassNonMaxSuppression/strided_slice_29/stack:0
Postprocessor/BatchMultiClassNonMaxSuppression/map/while/MultiClassNonMaxSuppression/Shape_44:0
Postprocessor/BatchMultiClassNonMaxSuppression/map/while/MultiClassNonMaxSuppression/ClipToWindow_49/Area/split/split_dim:0
Postprocessor/BatchMultiClassNonMaxSuppression/map/whi

In [11]:
def run_inference_for_single_image(image, graph):
  with graph.as_default():
    with tf.Session() as sess:
      # Get handles to input and output tensors
      ops = tf.get_default_graph().get_operations()
      all_tensor_names = {output.name for op in ops for output in op.outputs}
      tensor_dict = {}
      for key in [
          'num_detections', 'detection_boxes', 'detection_scores',
          'detection_classes', 'detection_masks'
      ]:
        tensor_name = key + ':0'
        if tensor_name in all_tensor_names:
          tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(
              tensor_name)
      if 'detection_masks' in tensor_dict:
        # The following processing is only for single image
        detection_boxes = tf.squeeze(tensor_dict['detection_boxes'], [0])
        detection_masks = tf.squeeze(tensor_dict['detection_masks'], [0])
        # Reframe is required to translate mask from box coordinates to image coordinates and fit the image size.
        real_num_detection = tf.cast(tensor_dict['num_detections'][0], tf.int32)
        detection_boxes = tf.slice(detection_boxes, [0, 0], [real_num_detection, -1])
        detection_masks = tf.slice(detection_masks, [0, 0, 0], [real_num_detection, -1, -1])
        detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
            detection_masks, detection_boxes, image.shape[1], image.shape[2])
        detection_masks_reframed = tf.cast(
            tf.greater(detection_masks_reframed, 0.5), tf.uint8)
        # Follow the convention by adding back the batch dimension
        tensor_dict['detection_masks'] = tf.expand_dims(
            detection_masks_reframed, 0)
      image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')

      # Run inference
      output_dict = sess.run(tensor_dict,
                             feed_dict={image_tensor: image})

      # all outputs are float32 numpy arrays, so convert types as appropriate
      output_dict['num_detections'] = int(output_dict['num_detections'][0])
      output_dict['detection_classes'] = output_dict[
          'detection_classes'][0].astype(np.uint8)
      output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
      output_dict['detection_scores'] = output_dict['detection_scores'][0]
      if 'detection_masks' in output_dict:
        output_dict['detection_masks'] = output_dict['detection_masks'][0]
  return output_dict

In [16]:
def getcocodict(output_dict, image_path, im_height, im_width):
    x = output_dict['detection_classes']
    detection_boxes_output = output_dict['detection_boxes']
    detection_scores_output =output_dict['detection_scores']
    person_indices = np.where(x == 1)
    p_detection_boxes = detection_boxes_output[person_indices]
    p_detection_scores = detection_scores_output[person_indices]
    ymin = p_detection_boxes[np.where(max(p_detection_scores))][0][0]
    xmin = p_detection_boxes[np.where(max(p_detection_scores))][0][1]
    ymax = p_detection_boxes[np.where(max(p_detection_scores))][0][2]
    xmax = p_detection_boxes[np.where(max(p_detection_scores))][0][3]

    (left, right, top, bottom) = (xmin * im_width, xmax * im_width,
                                      ymin * im_height, ymax * im_height)

    bbox = [left, top, right - left, bottom - top]
    single_image_dict = {}
    single_image_dict["image_id"] = image_path.split('/')[-1]
    single_image_dict["category_id"] = 1
    single_image_dict["bbox"] = bbox
    single_image_dict["score"] = float(max(p_detection_scores))
    return single_image_dict

In [17]:
def getcocovaldict(image_path, im_height, im_width):
    image_dict = {'license': 3, 'file_name': '00000032.jpg', 'coco_url': 
                  'http://images.cocodataset.org/val2017/000000394940.jpg', 'height': 360, 'width': 640, 
                  'date_captured': '203213-11-24 13:47:05', 
                  'flickr_url': 'http://farm9.staticflickr.com/8227/8566023505_e9e9f997bc_z.jpg', 'id': 32}
    image_dict['file_name'] = image_path.split('/')[-1]
    image_dict['height'] = im_height
    image_dict['width'] = im_width
    image_dict['id'] = image_path.split('/')[-1]
    return image_dict

In [19]:
detection_list = []
valimage_dict = {}
for image_path in TEST_IMAGE_PATHS:
    image = Image.open(image_path)
    # the array based representation of the image will be used later in order to prepare the
    # result image with boxes and labels on it.
    image_np = load_image_into_numpy_array(image)
    # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
    image_np_expanded = np.expand_dims(image_np, axis=0)
    # Actual detection.
    output_dict = run_inference_for_single_image(image_np_expanded, detection_graph)
    detection_list.append(getcocodict(output_dict, image_path, image_np.shape[0], image_np.shape[1]))
    valimage_dict[image_path.split('/')[-1]] = getcocovaldict(image_path, image_np.shape[0], image_np.shape[1])

In [20]:
with open('human_detection.json', 'w') as outfile:
    json.dump(detection_list, outfile)

with open('valimage_dict.json', 'w') as outfile:
    json.dump(valimage_dict, outfile)

TypeError: dump() missing 1 required positional argument: 'fp'

In [ ]:
print('End Here!')

In [ ]:
for image_path in TEST_IMAGE_PATHS:
    image = Image.open(image_path)
    # the array based representation of the image will be used later in order to prepare the
    # result image with boxes and labels on it.
    image_np = load_image_into_numpy_array(image)
    # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
    image_np_expanded = np.expand_dims(image_np, axis=0)
    # Actual detection.
    output_dict = run_inference_for_single_image(image_np_expanded, detection_graph)
    # Visualization of the results of a detection.
    vis_util.visualize_boxes_and_labels_on_image_array(
      image_np,
      output_dict['detection_boxes'],
      output_dict['detection_classes'],
      output_dict['detection_scores'],
      category_index,
      instance_masks=output_dict.get('detection_masks'),
      use_normalized_coordinates=True,
      line_thickness=8)
    plt.figure(figsize=IMAGE_SIZE)
    plt.imshow(image_np)

In [ ]:
%matplotlib inline
image = Image.open(TEST_IMAGE_PATHS[32])
  # the array based representation of the image will be used later in order to prepare the
  # result image with boxes and labels on it.
image_np = load_image_into_numpy_array(image)
  # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
image_np_expanded = np.expand_dims(image_np, axis=0)
  # Actual detection.
output_dict = run_inference_for_single_image(image_np_expanded, detection_graph)
  # Visualization of the results of a detection.
vis_util.visualize_boxes_and_labels_on_image_array(
      image_np,
      output_dict['detection_boxes'],
      output_dict['detection_classes'],
      output_dict['detection_scores'],
      category_index,
      instance_masks=output_dict.get('detection_masks'),
      use_normalized_coordinates=True,
      line_thickness=8)
plt.figure(figsize=IMAGE_SIZE)
plt.imshow(image_np)

In [ ]:
print(output_dict)

In [ ]:
print(getcocodict(output_dict, TEST_IMAGE_PATHS[32]))

In [ ]:
print(single_image_dict)

In [ ]:
import pycocotools


In [ ]:
import json
with open('data.json', 'w') as outfile:
    json.dump([single_image_dict], outfile)